In [5]:
import pandas as pd
import itertools

# Function to convert American odds to decimal odds
def american_to_decimal(odds):
    odds = float(odds)
    if odds < 0:
        return 1 + (100 / -odds)
    else:
        return 1 + (odds / 100)

# Function to convert decimal odds to American odds
def decimal_to_american(odds):
    odds = float(odds)
    if odds >= 2:
        return (odds - 1) * 100
    else:
        return -100 / (odds - 1)

# Function to calculate implied probability from American odds
def american_to_implied_prob(odds):
    odds = float(odds)
    if odds < 0:
        return -odds / (-odds + 100)
    else:
        return 100 / (odds + 100)

# Read the CSV file
df = pd.read_csv('Next30_Odds.csv')

# List to hold bets on favorites with odds between -500 and -2000
favorite_bets = []

# Process each row to identify favorites
for index, row in df.iterrows():
    try:
        odds1 = float(row['odds1'])
        odds2 = float(row['odds2'])
    except ValueError:
        continue  # Skip rows with invalid odds

    team1 = row['team1']
    team2 = row['team2']
    event = row['sport'] + " " + row['event_time']

    # Identify favorite and underdog, calculate discrepancy
    if odds1 < odds2:
        favorite_odds = odds1
        underdog_odds = odds2
        favorite_team = team1
    else:
        favorite_odds = odds2
        underdog_odds = odds1
        favorite_team = team2

    # Check if favorite odds are within the specified range
    if favorite_odds < 0 and -2000 <= favorite_odds <= -500:
        # Calculate implied probabilities
        favorite_implied_prob = american_to_implied_prob(favorite_odds)
        underdog_implied_prob = american_to_implied_prob(underdog_odds)
        # Calculate discrepancy
        discrepancy = favorite_implied_prob - underdog_implied_prob
        # Store the bet with discrepancy
        favorite_bets.append({
            'event': event,
            'team': favorite_team,
            'odds': favorite_odds,
            'discrepancy': discrepancy
        })

# Generate all combinations of bets (2 to 5-team parlays)
parlays = []

for r in range(2, 6):  # You can adjust the range as needed
    combinations = itertools.combinations(favorite_bets, r)
    for combo in combinations:
        # Ensure all events are unique to avoid conflicting bets
        event_names = [bet['event'] for bet in combo]
        if len(set(event_names)) < len(event_names):
            continue  # Skip combinations with multiple bets from the same event

        # Calculate combined decimal odds and implied probability
        combined_decimal_odds = 1.0
        combined_prob = 1.0
        total_discrepancy = 0.0
        events = []
        teams = []
        odds_list = []
        for bet in combo:
            decimal_odds = american_to_decimal(bet['odds'])
            implied_prob = american_to_implied_prob(bet['odds'])
            combined_decimal_odds *= decimal_odds
            combined_prob *= implied_prob
            total_discrepancy += bet['discrepancy']
            events.append(bet['event'])
            teams.append(bet['team'])
            odds_list.append(bet['odds'])
        # Convert combined decimal odds to American odds
        combined_american_odds = decimal_to_american(combined_decimal_odds)
        # Check if combined odds are between +100 and +200
        if 100 <= combined_american_odds <= 200:
            # Calculate expected value
            payout = combined_decimal_odds - 1  # Profit per $1 bet
            expected_value = (combined_prob * payout) - ((1 - combined_prob) * 1)
            parlays.append({
                'parlay': list(zip(events, teams, odds_list)),
                'combined_odds': combined_american_odds,
                'implied_probability': combined_prob,
                'expected_value': expected_value,
                'total_discrepancy': total_discrepancy
            })

# Sort parlays from most likely to least likely based on implied probability and discrepancy
sorted_parlays = sorted(parlays, key=lambda x: (-x['implied_probability'], -x['total_discrepancy']))

# Output the results to a file
with open('parlay_results.txt', 'w') as f:
    for parlay in sorted_parlays:
        f.write("Parlay:\n")
        for event, team, odds in parlay['parlay']:
            f.write(f"  Event: {event}, Team: {team}, Odds: {odds}\n")
        f.write(f"Combined Odds (American): {parlay['combined_odds']:+.2f}\n")
        f.write(f"Implied Probability: {parlay['implied_probability'] * 100:.2f}%\n")
        f.write(f"Total Discrepancy: {parlay['total_discrepancy']:.4f}\n")
        f.write(f"Expected Value per $1 bet: ${parlay['expected_value']:.2f}\n")
        f.write("-----\n")

print("Parlay results have been written to 'parlay_results.txt'.")

FileNotFoundError: [Errno 2] No such file or directory: 'Next_30Odds.csv'